## Creating Audio File

Checkout https://beta.elevenlabs.io/history for full log of audio creation

In [26]:
name_of_voice = "Bella"
text = "I love you"
file_name = "./temp"
mp3_file_name = file_name + ".mp3"
wav_file_name = file_name + ".wav"

In [23]:
# Get voice ID
import requests

r1 = requests.get('https://api.elevenlabs.io/v1/voices')
resp = r1.json()
rid = ""
for voice in resp["voices"]:
    if voice["name"] == name_of_voice:
        print(voice)
        rid = voice["voice_id"]
        break;
rid

{'voice_id': 'EXAVITQu4vr4xnSDxMaL', 'name': 'Bella', 'samples': None, 'category': 'premade', 'fine_tuning': {'is_allowed_to_fine_tune': False, 'fine_tuning_requested': False, 'finetuning_state': 'not_started', 'verification_attempts_count': 0}, 'labels': {}, 'preview_url': 'https://storage.googleapis.com/eleven-public-prod/premade/voices/EXAVITQu4vr4xnSDxMaL/04365bce-98cc-4e99-9f10-56b60680cda9.mp3', 'available_for_tiers': [], 'settings': None}


'EXAVITQu4vr4xnSDxMaL'

In [24]:
# Get audio file, minimize runs cost money
import json

url = "https://api.elevenlabs.io/v1/text-to-speech/{voice_id}".format(voice_id=rid)
print(url)
headers = {'xi-api-key': "aa42307ce6bfbefcfd2abd4d8634127b"}
payload = {
  "text": text,
  "voice_settings": {
    "stability": 0.3,
    "similarity_boost": 0.75
  }
}

r2 = requests.post(url, data=json.dumps(payload), headers=headers)
r2

https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL


In [25]:
# Save and play audio file
import os

with open(mp3_file_name, 'wb') as f:
    f.write(r2.content)

os.system("mpg123 " + mp3_file_name)

High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.31.2; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Directory: ./
Playing MPEG stream 1 of 1: temp.mp3 ...

MPEG 1.0 L III cbr64 44100 mono

[0:00] Decoding of temp.mp3 finished.


0

In [34]:
# convert mp3 to wav                                                            
from pydub import AudioSegment
from pydub.playback import play


sound = AudioSegment.from_mp3(mp3_file_name)
sound.export(wav_file_name, format="wav")

# test sound
song = AudioSegment.from_wav(wav_file_name)
play(song)

Input #0, wav, from '/var/folders/g8/b65pczf97pb17p1_zrldsm500000gn/T/tmpt9m7e2pp.wav':
  Duration: 00:00:00.84, bitrate: 706 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 1 channels, s16, 705 kb/s


## Generate phonemes
- Uses https://github.com/m-bain/whisperX

In [45]:
# no extension, must be wav
import os
cwd = os.getcwd() #pwd
print(cwd)

audio_file_name = wav_file_name
base_audio_file_name = os.path.basename(audio_file_name)
print("File name:", audio_file_name)
print("Base name:", base_audio_file_name)

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/scripts
File name: ./temp.wav
Base name: temp.wav


In [46]:
import whisperx

device = "cpu" 
# transcribe with original whisper
model = whisperx.load_model("large", device)

In [49]:
result = model.transcribe(audio_file_name)

print(result["segments"]) # before alignment

# load alignment model and metadata
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

# align whisper output
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file_name, device)

# print(result_aligned["segments"]) # after alignment
result_aligned["word_segments"]

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/whisperx/transcribe.py:83: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 0.64, 'text': ' I love you.', 'tokens': [50364, 286, 959, 291, 13, 50396], 'temperature': 0.0, 'avg_logprob': -0.5376458849225726, 'compression_ratio': 0.5789473684210527, 'no_speech_prob': 0.029172642156481743}]


/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/whisperx/alignment.py:302: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  char_segments_arr = per_seg_grp.apply(lambda x: x.reset_index(drop = True)).reset_index()


[{'text': 'I', 'start': 0.1032258064516129, 'end': 0.16516129032258065},
 {'text': 'love', 'start': 0.2064516129032258, 'end': 0.3716129032258064},
 {'text': 'you.', 'start': 0.4129032258064516, 'end': 0.49548387096774194}]

In [52]:
import nltk

try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

In [53]:
import re
from functools import lru_cache
from itertools import product as iterprod

def cleanWord(word):
    regex = r"[^\w\']"
    return re.sub(regex, "", word).lower()

def wordbreak(s):
    s = cleanWord(s)
    if s in arpabet:
        return arpabet[s]
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabet[pre], wordbreak(suf))]
    return None

# Example words: 
for w in ["that's", "Help! ", "multiverse"]:
    print(wordbreak(w))

[['DH', 'AE1', 'T', 'S']]
[['HH', 'EH1', 'L', 'P']]
[['M', 'AH1', 'L', 'T', 'IY0', 'V', 'ER1', 'S']]


In [54]:
# {'text': 'Good', 'start': 0.10120481927710842, 'end': 0.32385542168674697},
# INTO
# [
#     {
#         "word": "sil",
#         "phones": [
#             {
#                 "ph": "SIL",
#                 "bg": 0,
#                 "ed": 126
#             }
#         ]
#     },
#     {
#         "word": "YOU",
#         "phones": [
#             {
#                 "ph": "Y",
#                 "bg": 126,
#                 "ed": 138
#             },
#             {
#                 "ph": "UW",
#                 "bg": 138,
#                 "ed": 141
#             }
#         ]
#     },
from math import trunc

def makeSegment(word, phones, start, end):
    total = end - start
    time_per_phone = trunc(total / len(phones))
#     print(total, time_per_phone)
    expanded_phones = [{"ph": phone, "bg": start + (i * time_per_phone), "ed": start + ((i + 1) * time_per_phone)} for i, phone in enumerate(phones)]
    return {"word": word, "phones": expanded_phones}

def makeSil(start, end):
    return {"word": "sil", "phones": [{"ph": "SIL", "bg": start, "ed": end}]}
  
all_phonemes = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    #Get rid of numbers in phones
    phonemes = ["".join(filter(lambda x: x.isalpha(), phone)) for phone in wordbreak(word)[0]]
    print(phonemes)
    start_time = trunc(segment["start"] * 100)
    end_time = trunc(segment["end"] * 100)
    
    if len(all_phonemes) > 0:
        last = all_phonemes[-1]
        last_end = last["phones"][-1]["ed"]
        if start_time - last_end > 5:
            sil = makeSil(last_end, start_time)
            all_phonemes.append(sil)
        else:
            start_time = last_end
    else:
        sil = makeSil(0, start_time)
        all_phonemes.append(sil)
        
    segment = makeSegment(word, phonemes, start_time, end_time)
    all_phonemes.append(segment)

all_phonemes

['AY']
['L', 'AH', 'V']
['Y', 'UW']


[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 10}]},
 {'word': 'I', 'phones': [{'ph': 'AY', 'bg': 10, 'ed': 16}]},
 {'word': 'love',
  'phones': [{'ph': 'L', 'bg': 16, 'ed': 23},
   {'ph': 'AH', 'bg': 23, 'ed': 30},
   {'ph': 'V', 'bg': 30, 'ed': 37}]},
 {'word': 'you.',
  'phones': [{'ph': 'Y', 'bg': 37, 'ed': 43},
   {'ph': 'UW', 'bg': 43, 'ed': 49}]}]

In [139]:
import json

file_name = "../samples/phonemes/{name}.json".format(name=audio_file_name)
print("Writing to ", file_name)
f1 = open(file_name, 'w')
json.dump(all_phonemes, f1)

Writing to  ../samples/phonemes/kitten_full.json


In [55]:
all_phonemes

[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 10}]},
 {'word': 'I', 'phones': [{'ph': 'AY', 'bg': 10, 'ed': 16}]},
 {'word': 'love',
  'phones': [{'ph': 'L', 'bg': 16, 'ed': 23},
   {'ph': 'AH', 'bg': 23, 'ed': 30},
   {'ph': 'V', 'bg': 30, 'ed': 37}]},
 {'word': 'you.',
  'phones': [{'ph': 'Y', 'bg': 37, 'ed': 43},
   {'ph': 'UW', 'bg': 43, 'ed': 49}]}]